<a href="https://colab.research.google.com/github/Vasanta2002/Sprint_5/blob/main/JDS_SHR_213_guided_project_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BloomTech Data Science

---


# Ridge Regression
- Do one-hot encoding of categorical features
- Use scikit-learn to fit Ridge Regression models



In [ ]:
# Model Building

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# For running this notebook either locally or in colab
import sys

In [ ]:
%%capture
# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/bloominstituteoftechnology/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# For encoding categorical data
from category_encoders import OneHotEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# I. Wrangle Data

Create **wrangle function** for **reproducibility**.

In [ ]:
def wrangle(filepath):
  df = pd.read_csv(filepath)

  # Remove outlier observations
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
          (df['price'] <= np.percentile(df['price'], 99.5)) & 
          (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
          (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
          (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
          (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

  # Drop high-cardinality categorical variables


  return df

df = wrangle(DATA_PATH+'apartments/renthop-nyc.csv')

In [ ]:
# threshold = 10
# cols_to_drop = []
# for col in df:
#   if df[col].dtype == 'object':
#     if df[col].nunique() > threshold:
#       cols_to_drop.append(col)

# df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
  cutoff = 90
  drop_cols = [col for col in df.select_dtypes('object').columns 
              if df[col].nunique() > cutoff]
  df.drop(columns=drop_cols, inplace=True)

# II. Split Data

Split **target vector** from **feature matrix**.

In [ ]:
target =
y = df[target]
X = df.drop(columns=target)

Split data into **training** and **test** sets.

(Use data from April & May 2016 to train. Use data from June 2016 to test.)

In [ ]:
X.index.max(), X.index.min()

In [ ]:
# Several masks to choose from

# mask = 
# mask = 


In [ ]:
# or apply the mask based on cut-off given below
cutoff =

mask =

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

# III. Establish Baseline

In [ ]:
y_pred = [y_train.mean()] * len(y_train)
print('Mean price:', y_train.mean())
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

# IV. Build Models

In [ ]:
# Step 1: Import your transformer class
# We did above ☝️

# Step 2: Insantiate your transformer

# Step 3: Fit your transformer

# Step 4: Transform 


**Model 1:** Regular ol' linear regression

In [ ]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

**Model 2:** Ridge regression (regularization)

# V. Check Metrics

In [ ]:
print('LR training MAE:', mean_absolute_error(y_train, model_lr.predict(XT_train))) 
print('LR test MAE:', mean_absolute_error(y_test, model_lr.predict(XT_test)))

In [ ]:
print('Ridge training MAE:', mean_absolute_error(y_train, model_r.predict(XT_train)))
print('Ridge test MAE:', mean_absolute_error(y_test, model_r.predict(XT_test)))

# Communicate results

In [ ]:
coefficients = 
features =

feat_imp = pd.Series(coefficients, index=features).sort_values(key=abs)
feat_imp.tail(10).plot(kind='barh')

plt.xlabel('Coefficient [$]')
plt.ylabel('Feature')
plt.title('Coefficients for Linear Regression');

In [ ]:
coefficients = 
features =

feat_imp = pd.Series(coefficients, index=features).sort_values(key=abs)
feat_imp.tail(10).plot(kind='barh')

plt.xlabel('Coefficient [$]')
plt.ylabel('Feature')
plt.title('Coefficients for Ridge Regression');


In [ ]:
# import seaborn as sns
# sns.set(rc={'figure.figsize':(15,15)})
# sns.scatterplot('longitude', 'latitude', hue='price', data=df);